In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from classes.classes import ColorManager
from classes.classes import E2EModelConfig
from classes.classes import FlatModelActCardModelConfig, QPPModelActCardsConfig, ZeroShotModelActCardConfig, \
    DACEModelActCardConfig, DACEModelConfig, FlatModelConfig
from classes.classes import MODEL_CONFIGS
from classes.classes import ZeroShotModelConfig, QPPNetModelConfig, ScaledPostgresModelConfig
from classes.paths import LocalPaths
from classes.workloads import EvalWorkloads
from classes.workloads import JoinOrderEvalWorkload
from cross_db_benchmark.datasets.datasets import Database
from evaluation.evaluation_metrics import QError
from evaluation.evaluation_metrics import SelectedRuntime
from evaluation.evaluation_metrics import SpearmanCorrelation
from evaluation.utils import draw_bushiness
from evaluation.utils import get_model_results, draw_predictions, draw_metric
from evaluation.evaluation_metrics import MissedPlansFraction, MaxOverestimation, MaxUnderestimation
from classes.classes import ACT_CARD_MODEL_CONFIGS
from matplotlib.lines import Line2D

sns.set_theme(style="whitegrid", font_scale=1.8)
fontsize = 14

----
## 2. Join Order Examples

In [ ]:
path = LocalPaths().data / "plots" / "join_order_examples.pdf"
mosaic = """AAAABBEE
            AAAACCFF
            AAAADDGG"""
folder = "join_order_full"
grid_spec = {'height_ratios': [1,1,1], 'wspace': 3, 'hspace': 0.3}

workloads = [JoinOrderEvalWorkload(database=Database("imdb"), folder=folder, wl_name="job_light_33", num_tables=4)]

title_1 =("SELECT COUNT(*) FROM title, movie_keyword, movie_companies\n"
          "WHERE title.id=movie_keyword.movie_id "
          "AND title.id=movie_companies.movie_id AND title.prod_year>1950;")

fig = plt.figure(figsize=(11 * len(workloads) , 3), dpi=100)
figures= fig.subfigures(nrows=1, ncols=len(workloads), wspace=-0.1, hspace=-0.05)

for idx, (workload, figure) in enumerate(zip(workloads, [figures])):
    subplots = figure.subplot_mosaic(mosaic, gridspec_kw=grid_spec).values()
    [prediction, q_error, runtime, missed_plans, spearmans, underest, overest] = subplots
    results = get_model_results(workload, MODEL_CONFIGS)
    # sort results by real runtime
    results = results.sort_values(by='runtime')
    draw_predictions(workload, results, MODEL_CONFIGS, prediction, fontsize=fontsize)
    draw_metric(results, MODEL_CONFIGS, q_error, QError(), fontsize)
    draw_metric(results, MODEL_CONFIGS, spearmans, SpearmanCorrelation(), fontsize)
    draw_metric(results, MODEL_CONFIGS, missed_plans, MissedPlansFraction(), fontsize)
    draw_metric(results, MODEL_CONFIGS, overest, MaxOverestimation(), fontsize)
    draw_metric(results, MODEL_CONFIGS, underest, MaxUnderestimation(), fontsize)
    draw_metric(results, MODEL_CONFIGS, runtime, SelectedRuntime(display_name="Selected\nRuntime(s)"), fontsize)
    prediction.set_xlabel('Join Enumeration', fontsize=fontsize)
    q_error.set_ylim(1, 3)
    q_error.set_yticks([1, 2, 3], labels=[1, 2, 3], fontsize=fontsize)
    q_error.minorticks_off()
    runtime.set_ylim(0, 6)
    underest.set_yticks([1, 3, 5], labels=[1, 3, 5], fontsize=fontsize)
    underest.minorticks_off()
    overest.set_yticks([1, 5, 10], labels=[1, 5, 10], fontsize=fontsize)
    overest.minorticks_off()

    figure.suptitle(title_1, fontsize=fontsize*0.8, fontproperties={'family':'monospace'}, y=1.03, horizontalalignment='center')
    runtime.axhline(y=results['runtime'].min(), linestyle='--', color='black', linewidth=2, zorder=100)
    runtime.annotate(text='Optimal\nRuntime',
                     xy=(1.05, 0.21),
                     xycoords='axes fraction',
                     fontsize=fontsize * 0.75,
                     ha='left',
                     va='bottom')

    for plot, letter in zip(list(subplots), "ABCDEFG"):
        plot.annotate(
            letter,
            xy=(0.05, 0.95), 
            xycoords='axes fraction', 
            fontsize=9, 
            ha='center', 
            va='center', 
            bbox=dict(boxstyle='circle,pad=0.2', edgecolor='black', facecolor='white'))
        
        
legend_handles = [mpatches.Patch(color=model_config.color(), label=model_config.name.DISPLAY_NAME) for model_config in MODEL_CONFIGS]
for p in legend_handles:
    p.set_edgecolor('black')
legend_handles.append(Line2D([0], [0], color='black', lw=1, linestyle='-', label='Real Runtime'))

legend = q_error.legend(handles=legend_handles, 
                        fontsize=fontsize,
                        ncol=1, 
                        loc='center left', 
                        bbox_to_anchor=(-5, -0.8),
                        labelspacing=0.3,
                        edgecolor='white')
for line in legend.get_lines():
    line.set_linewidth(6.0)
    
fig.align_labels()
plt.savefig(path, bbox_inches='tight')

In [ ]:
results

In [ ]:
from classes.classes import QueryFormerModelConfig, E2EModelConfig
metric = SpearmanCorrelation()
model_configs = [ScaledPostgresModelConfig(), QueryFormerModelConfig(), E2EModelConfig()]
extract = pd.DataFrame({
        model.name.DISPLAY_NAME: metric.evaluate_metric(preds=results[results["model"] == model.name.DISPLAY_NAME]["prediction"],
                                                        labels=results[results["model"] == model.name.DISPLAY_NAME]['runtime']) for model in model_configs},
        index=[metric.metric_name]).T
extract

In [ ]:
results["runtime"].max()

----
## 3. Bushiness Examples

In [ ]:
path = LocalPaths().data / "plots" / "join_order_bushiness.pdf"
workloads = [JoinOrderEvalWorkload(database=Database("baseball"), folder=folder, wl_name="baseball_14", num_tables=4),
             JoinOrderEvalWorkload(database=Database("imdb"), folder=folder, wl_name="job_light_34", num_tables=4),
             JoinOrderEvalWorkload(database=Database("tpc_h"), folder=folder, wl_name="tpc_h_88", num_tables=4),
             JoinOrderEvalWorkload(database=Database("baseball"), folder=folder, wl_name="baseball_58", num_tables=4),
             JoinOrderEvalWorkload(database=Database("imdb"), folder=folder, wl_name="job_light_33", num_tables=4),
             JoinOrderEvalWorkload(database=Database("tpc_h"), folder=folder, wl_name="tpc_h_117", num_tables=4)]

fig, axs = plt.subplots(2,3, figsize=(20, 6), dpi=100)

for workload, ax in zip(workloads, axs.flatten()):
    results = get_model_results(workload, MODEL_CONFIGS)
    draw_bushiness(results, MODEL_CONFIGS, ax)

axs[0][0].set_ylabel("Q-Error")
axs[1][0].set_ylabel("Q-Error")
fig.align_labels()
plt.savefig(path, bbox_inches='tight')

----
## 3. Join Order (Full Benchmark)

In [ ]:
from classes.classes import MSCNModelConfig, QueryFormerModelConfig

path = LocalPaths().data / "plots" / "join_order_full.pdf"

TEST_MODEL_CONFIGS = [
    ScaledPostgresModelConfig(),
    FlatModelConfig(),
    MSCNModelConfig(),
    E2EModelConfig(),
    ZeroShotModelConfig(),
    QPPNetModelConfig(),
    QueryFormerModelConfig(),
    DACEModelConfig(),
    FlatModelActCardModelConfig(),
    QPPModelActCardsConfig(),
    ZeroShotModelActCardConfig(),
    DACEModelActCardConfig()
]

for plot, model_configs in zip(["join_order_full", "join_order_full_act_cards"], [MODEL_CONFIGS, TEST_MODEL_CONFIGS]):
    path = LocalPaths().data / "plots" / f"{plot}.pdf"
    
    spearman_df = pd.DataFrame()
    runtime_df = pd.DataFrame()
    missed_plans_df = pd.DataFrame()
    overest_df  = pd.DataFrame()
    underest_df = pd.DataFrame()
    minimal_runtimes = []
    
    for workload in EvalWorkloads.FullJoinOrder.imdb:
        results = get_model_results(workload, model_configs)
        for model in model_configs:
            model_results = results[results["model"] == model.name.DISPLAY_NAME]
            spearman_corr = SpearmanCorrelation().evaluate_metric(preds=model_results["prediction"], 
                                                                  labels=model_results['runtime'])
            spearman_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = spearman_corr
            selected_runtime = SelectedRuntime().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
            runtime_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = selected_runtime
            missed_plans = MissedPlansFraction().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
            missed_plans_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = missed_plans
            overest = MaxOverestimation().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
            overest_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = overest
            underest = MaxUnderestimation().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
            underest_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = underest
        minimal_runtimes.append(model_results["label"].min())
    
    # Create the boxplot and barplot
    fig, (ax2, ax3, ax1, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 3))
    
    sns.boxplot(data=spearman_df.T, palette=ColorManager.COLOR_PALETTE, ax=ax1)
    #ax1.set_title('Spearman Correlation over JOB-Light Permutations')
    ax1.set_xlabel('')
    ax1.set_title('Spearman Correlation', fontsize=fontsize)
    ax1.xaxis.set_ticklabels([])
    ax1.set_ylim(-1, 1)
    ax1.tick_params(axis='y', which='major', pad=0, labelsize=fontsize)
    
    sns.barplot(data=runtime_df.sum(axis=1).reset_index(), x='index', hue='index', palette=ColorManager.COLOR_PALETTE, y=0, ax=ax2, edgecolor='black')
    #ax2.set_title('Total Runtime for Each Model')
    ax2.set_xlabel('')
    ax2.set_title('Total Runtime (s)', fontsize=fontsize)
    ax2.xaxis.set_ticklabels([])
    ax2.set_ylabel("")
    ax2.axhline(y=sum(minimal_runtimes), color='black', linestyle='--')
    ax2.tick_params(axis='y', which='major', pad=0, labelsize=fontsize)
    
    sns.boxplot(data=missed_plans_df.T, palette=ColorManager.COLOR_PALETTE, ax=ax3)
    ax3.set_xlabel('')
    ax3.set_title('Surpassed Plans (%)', fontsize=fontsize)
    ax3.xaxis.set_ticklabels([])
    ax3.tick_params(axis='y', which='major', pad=0,  labelsize=fontsize)
    
    sns.boxplot(data=underest_df.T, palette=ColorManager.COLOR_PALETTE, ax=ax4)
    ax4.set_xlabel('')
    ax4.set_title('Underestimation', fontsize=fontsize)
    ax4.xaxis.set_ticklabels([])
    ax4.set_ylim(0.9, 25)
    ax4.set_yscale('log')
    ax4.tick_params(axis='y', which='major', pad=0,  labelsize=fontsize)
    
    sns.boxplot(data=overest_df.T, palette=ColorManager.COLOR_PALETTE, ax=ax5)
    ax5.set_xlabel('')
    ax5.set_title('Overestimation', fontsize=fontsize)
    ax5.xaxis.set_ticklabels([])
    ax5.set_ylim(0.9, 25)
    ax5.set_yscale('log')
    ax5.tick_params(axis='y', which='major', pad=0,  labelsize=fontsize)
    
    for ax in (ax2, ax3, ax1, ax4, ax5):
        ax.axvspan(xmin=7.5, xmax=11.5, alpha=0.1, color='gray')
    
    # Create legend patches
    legend_patches = [mpatches.Patch(color=model_config.color(), label=model_config.name.DISPLAY_NAME) for model_config in model_configs]
    for p in legend_patches:
        p.set_edgecolor('black')
    legend_patches.insert(0, Line2D([0], [0], color='black', lw=2, linestyle='--', label='Optimal Runtime'))
    legend_patches.insert(9, plt.Line2D([], [], linewidth=0))
    ax2.legend(handles=legend_patches, 
               loc='center right', 
               bbox_to_anchor=(-0.2, 0.5), 
               edgecolor='white', 
               labelspacing=0.1, 
               fontsize=fontsize * 0.8)
    
    #for i in ax2.containers:
    #    ax2.bar_label(i,fmt='%.0f', label_type='edge', fontsize=fontsize*0.7)
        
    fig.align_labels()
    plt.savefig(path, bbox_inches='tight')

In [ ]:
model_configs = ACT_CARD_MODEL_CONFIGS

spearman_df = pd.DataFrame()
runtime_df = pd.DataFrame()
missed_plans_df = pd.DataFrame()
overest_df  = pd.DataFrame()
underest_df = pd.DataFrame()
q_error_df = pd.DataFrame()

minimal_runtimes = []
num_tables = []
for workload in EvalWorkloads.FullJoinOrder.imdb:
    results = get_model_results(workload, model_configs)
    for model in model_configs:
        model_results = results[results["model"] == model.name.DISPLAY_NAME]
        spearman_corr = SpearmanCorrelation().evaluate_metric(preds=model_results["prediction"], 
                                                              labels=model_results['runtime'])
        spearman_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = spearman_corr
        q_error = QError().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
        q_error_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = q_error
        selected_runtime = SelectedRuntime().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
        runtime_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = selected_runtime
        missed_plans = MissedPlansFraction().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
        missed_plans_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = missed_plans
        overest = MaxOverestimation().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
        overest_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = overest
        underest = MaxUnderestimation().evaluate_metric(preds=model_results["prediction"], labels=model_results['runtime'])
        underest_df.loc[model.name.DISPLAY_NAME, workload.get_workload_name()] = underest
    minimal_runtimes.append(model_results["label"].min())
    num_tables.append(workload.num_tables)
        
# Merge numtables to spearmans:
spearman_df = spearman_df.T
spearman_df['num_tables'] = num_tables

q_error_df = q_error_df.T
q_error_df['num_tables'] = num_tables

In [ ]:
# Group dataframe by num tables and prepare the data for plotting
# Assuming q_error_df is already defined
# Remove rows that contain at least 4 NaN values
q_error_df = q_error_df.dropna(thresh=4)

# Group dataframe by num tables and calculate the median for each group
grouped = q_error_df.groupby('num_tables')

# Create a single DataFrame to store all the median data for plotting
median_data = []

for name, group in grouped:
    group = group.drop(columns='num_tables')
    median_group = group.median().reset_index()
    median_group.columns = ['Model', 'Q-Error']
    median_group['num_tables'] = name
    median_data.append(median_group)

# Concatenate all the median data into one DataFrame
median_data = pd.concat(median_data)

spearman_df = spearman_df.dropna(thresh=4)

# Group dataframe by num tables and calculate the median for each group
spearman_df = spearman_df.groupby('num_tables')

# Create a single DataFrame to store all the median data for plotting
spearman_df_collection = []

for name, group in grouped:
    group = group.drop(columns='num_tables')
    median_group = group.median().reset_index()
    median_group.columns = ['Model', 'Q-Error']
    median_group['num_tables'] = name
    spearman_df_collection.append(median_group)

# Concatenate all the median data into one DataFrame
spearman_df_collection = pd.concat(spearman_df_collection)


# Create the plot
fig, axs = plt.subplots(2, 1, figsize=(10, 5))
sns.barplot(data=median_data, x='num_tables', y='Q-Error', hue='Model', palette=ColorManager.COLOR_PALETTE, ci=None, ax=axs[0])
sns.barplot(data=spearman_df_collection, x='num_tables', y='Q-Error', hue='Model', palette=ColorManager.COLOR_PALETTE, ax=axs[1])
axs[0].set_ylim(1, 3.5)
#ax.set_yscale('log')

    
# Create legend
legend_patches = [mpatches.Patch(color=model_config.color(), label=model_config.name.DISPLAY_NAME) for model_config in model_configs]
for p in legend_patches:
    p.set_edgecolor('black')
    
ax.legend(handles=legend_patches, 
           loc='center right', 
           bbox_to_anchor=(-0.2, 0.5), 
           edgecolor='white', 
           labelspacing=0.3, 
           fontsize=fontsize)
    
plt.show()